<a href="https://colab.research.google.com/github/xdderekchen/ML/blob/master/data_import/Test_WEBAPI_ttdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demonstation of Web API of ttData

ttdata provides webservices for users to access the powerful block-chain based services.

This post has 2 purposes:
 1. Samples for API calls
 2. Testing of API and the webservice

# 1.0.  Step-by-Step run

In this version, we will test the following Web API functions
 * **(POST)** http://URL:port/register
 * **(POST)** http://URL:port/uploaddata
 * **(GET)**  http://URL:port/getshareddata

### Setup Server URL

In [0]:
baseURL = "http://108.48.52.183:7061/"   #production server
#baseURL = "http://ixinbuy.com:7061/"    #Azure Server
#baseURL = "http://ttdata.life:8080/"
baseURL = "http://ttdata.life:7061/"

DEVICE_ID = "XD_COVID19_V1"
DEVICE_ID = "XDCOVID19V512"

### register  (endpoint: /register)

In [19]:
import requests
import json

URL = baseURL +"register"

payload = {"device_id":DEVICE_ID, "category": "test"}

#Passing payload as dict
responce = requests.post(URL, data = payload)
print(responce.status_code)
print(responce.text)


200
true


### uploaddata (simple sample data) (endpoint: /uploaddata)

In [0]:
import requests
import json
URL = baseURL + "uploaddata"
print(URL)
# data_type": 2 for COVID19 data
payload = {
    "data_type": 2, 
    "device_id": DEVICE_ID, 
    "key": "date|province_state|county|country_region",
    "device_data": 
    [{  "date": "2001-05-02", "country_region": "US", "province_state": "New York","city": "Albany2",
        "last_update": "2020-03-12T10:43:02",
        "confirmed":222,"deaths": 2, "recovered": 0,
        "latitude":42.60060307, "longitude": -73.97723916
    },
    {  "date": "2002-05-02", "country_region": "US", "province_state": "New York", "city": "Albany2",
        "last_update": "2020-03-12T10:43:02",
        "confirmed":222, "deaths": 2, "recovered": 0,
        "latitude":42.60060307, "longitude": -73.97723916
    }
    ]
}

#Passing payload as dict
import time
ts = time.time()
responce = requests.post(URL, json = payload)
print(responce.status_code)
print(responce.text)
te = time.time()
print('%s  %2.2f sec' % ("uploaddata", (te - ts) ))

http://ttdata.life:7061/uploaddata
200
COVID19 data uploaded successfully
uploaddata  1.50 sec


### function to standarize names.

In [0]:
import numpy as np
import pandas as pd
import os 

def transform_names(df):
    '''
       need to have standard names
    '''
    df.columns = [x.lower() for x in df.columns]
    df.rename(columns={'province/state': 'province_state',
                   'admin2'            : 'county',
                   'country/region'    : 'country_region',
                   'last update'       : 'last_update',
                   "long_":          "longitude",
                   "long":          "longitude",
                   "lat":           "latitude",
                   "lat_":           "latitude",
                  }, inplace=True)
    
    if ("combined_key" in df.columns):
       df.drop(columns=['combined_key'], inplace=True)
       
    keycolumn = "country_region|province_state|date"

    if ("county" in df.columns):
        keycolumn = "country_region|province_state|county|date"
  
    return (df, keycolumn)

#######################################################################
##### Testing
def run_test(date= "03-20-2020"):
    github_root = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports"
    file = os.path.join(github_root, date + ".csv")
    df = pd.read_csv(file ,dtype = {'FIPS': str, 'ZIP': str, "ZIP_CODE": str})
    df, key = transform_names(df)
    print(key)
    print(df.columns)

    row = df.iloc[1]
    print(json.loads(row.to_json()))

run_test("03-20-2020")  #Old format
run_test("03-26-2020")  #new format


country_region|province_state|date
Index(['province_state', 'country_region', 'last_update', 'confirmed',
       'deaths', 'recovered', 'latitude', 'longitude'],
      dtype='object')
{'province_state': None, 'country_region': 'Italy', 'last_update': '2020-03-20T17:43:03', 'confirmed': 47021, 'deaths': 4032, 'recovered': 4440, 'latitude': 41.8719, 'longitude': 12.5674}
country_region|province_state|county|date
Index(['fips', 'county', 'province_state', 'country_region', 'last_update',
       'latitude', 'longitude', 'confirmed', 'deaths', 'recovered', 'active'],
      dtype='object')
{'fips': '22001', 'county': 'Acadia', 'province_state': 'Louisiana', 'country_region': 'US', 'last_update': '2020-03-26 23:48:35', 'latitude': 30.2950649, 'longitude': -92.41419698, 'confirmed': 3, 'deaths': 0, 'recovered': 0, 'active': 0}


### Covid19data_to_ttdata

We will pull real data from John Hopkins and invoke the webservice call.

The source data is located at "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports"
    

In [0]:
from pprint import pprint 
import pandas as pd
import numpy as np
import os
import requests
import json

def covid19data_to_ttdata(source_file, URL, data_date, blocksize=0, waitsec=10):
    print("processing: ", source_file)
    df = pd.read_csv(source_file, dtype = {'FIPS': str, 'ZIP': str, "ZIP_CODE": str})
    df, key = transform_names(df)
    errorCount = 0
    
    totalsize = df.shape[0]
    print(totalsize)
    if blocksize == 0:
       blocksize = totalsize
    batches =[list(range(i, min(i+blocksize, totalsize))) for i in range(0,totalsize,blocksize) ]

    for aBatch in batches:
        json_data_list = []
        for i in aBatch:
            row = df.iloc[i]
            #row_as_dic = row.to_dict()
            device_data = {"date":data_date }
            device_data.update(json.loads(row.to_json()))
            json_data_list.append(device_data)
        
        print("\nFor batch from: ", aBatch[0], ", to:", aBatch[-1])
        payload = {
             "data_type": 2, 
             "device_id": DEVICE_ID,
             "key"      : key,
             "device_data": json_data_list
              }
        #print(payload)
        responce = requests.post(URL, json = payload)
        print(responce.elapsed.total_seconds())
        print(responce.status_code)
        print(responce.text)
        time.sleep(waitsec)
        

github_root = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports"


file = os.path.join(github_root, "03-28-2020" + ".csv")

import time
ts = time.time()
d = covid19data_to_ttdata( source_file= file, 
                           URL = baseURL + "uploaddata", 
                           data_date = "2020-03-28", 
                           blocksize = 200,
                           waitsec = 1)
te = time.time()
print('%s  %2.2f sec' % ("uploaddata", (te - ts) ))
 


processing:  https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-28-2020.csv
3430

For batch from:  0 , to: 199
1.638541
200
COVID19 data uploaded successfully

For batch from:  200 , to: 399
38.616032
200
COVID19 data uploaded successfully

For batch from:  400 , to: 599
34.307485
200
COVID19 data uploaded successfully

For batch from:  600 , to: 799
39.005344
200
COVID19 data uploaded successfully

For batch from:  800 , to: 999
50.721459
200
COVID19 data uploaded successfully

For batch from:  1000 , to: 1199
41.615733
200
COVID19 data uploaded successfully

For batch from:  1200 , to: 1399
47.829563
200
COVID19 data uploaded successfully

For batch from:  1400 , to: 1599
53.504604
200
COVID19 data uploaded successfully

For batch from:  1600 , to: 1799
47.102126
200
COVID19 data uploaded successfully

For batch from:  1800 , to: 1999
52.166635
200
COVID19 data uploaded successfully

For batch from:  2000 , to: 2199
51.3

### getshareddata (GET)

This is the method to retrieve data from server



In [0]:
import pandas as pd
import os
import requests
import json

query = {"data_type": 2, "device_id":DEVICE_ID , "from_date": "2020-04-02", "to_date":  "2020-04-03"}

req = requests.get(baseURL + "getshareddata", params=query)
print(req.url)
print(req.status_code)

#print(req.text)
data = req.json()
df = pd.DataFrame(data)

df
#df = df[~df['country_region'].isin(["China000", "China111"])]
#df = df.drop_duplicates(subset=["country_region", "province_state", "date" ], keep='last')
#df = df.query('country_region=="Italy"')

#df_agg=df.groupby(["country_region", "date"]).agg({'confirmed': ['sum'], 'deaths': ['sum'], 'recovered': ['sum']})
#df_agg.columns = ['confirmed', 'deaths', 'recovered']
#df_agg.reset_index(inplace=True)

#df_agg["fatality_rate"] = df_agg["deaths"] / df_agg["confirmed"]

#df_confirm = df_agg.sort_values(by='confirmed', ascending=False).head(120)

#df_confirm

#df[ (df["country_region"]=="US") & (df["province_state"] == "New York")]

http://ttdata.life:7061/getshareddata?data_type=2&device_id=XDCOVID19V5&from_date=2020-04-02&to_date=2020-04-03
200


,date,fips,county,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,valid
0,2020-04-02,36001,Albany,New York,US,4/2/20 23:25,42.600603,-73.977239,253,2,0,0,True
1,2020-04-03,36001,Albany,New York,US,2020-04-03 22:46:37,42.600603,-73.977239,267,2,0,0,True
2,2020-04-02,45001,Abbeville,South Carolina,US,4/2/20 23:25,34.223334,-82.461707,6,0,0,0,True
3,2020-04-02,29001,Adair,Missouri,US,4/2/20 23:25,40.190586,-92.600782,6,0,0,0,True
4,2020-04-02,51001,Accomack,Virginia,US,4/2/20 23:25,37.767072,-75.632346,10,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5180,2020-04-03,None,None,None,West Bank and Gaza,2020-04-03 22:46:20,31.952200,35.233200,194,1,21,172,True
5181,2020-04-03,None,None,None,Venezuela,2020-04-03 22:46:20,6.423800,-66.589700,153,7,52,94,True
5182,2020-04-03,None,None,None,Zimbabwe,2020-04-03 22:46:20,-19.015438,29.154857,9,1,0,8,True
5183,2020-04-03,None,None,None,Zambia,2020-04-03 22:46:20,-13.133897,27.849332,39,1,2,36,True


### visualization

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")

df_fatal_rate = df_confirm.sort_values(by='fatality_rate', ascending=False)

# Initialize the matplotlib figure
f, (ax1, ax2) = plt.subplots(figsize=(16, 14), ncols=2)


sns.barplot(x="confirmed", y="country_region", data=df_confirm,
            label="confirmed", color="b",  ax=ax1)
ax1.set_title("Top 20 countries by the number of confirmed cases")

sns.barplot(x="fatality_rate", y="country_region", data=df_fatal_rate,
            label="fatality rate", color="y",  ax=ax2)
ax2.set_title("Fatality Rate, for top 20 countries by the number of confirmed cases")

In [0]:
import plotly.express as px

df['province_state'] = df['province_state'].fillna(df['country_region'])
fig = px.scatter_mapbox(data_frame=df, lat="latitude", lon="longitude", hover_data=["confirmed", "deaths"],  
                        size="confirmed", zoom=2, height=1000, hover_name="province_state" )
fig.update_layout(mapbox_style="open-street-map")
fig.show()

# 2.0 Combined Code

The following code can be copied to a single python file. Then run it from a command line.

Please remember to set *interactive=False* in the code [
**exit(main(interactive=False))** ]

### Main Program


In [38]:
import requests
import json
import numpy as np
import pandas as pd
import os 
import time
from datetime import datetime
import logging
import argparse

def showtime(tstart):
    """
    Show runtime duration since tstart

    parameters
    ----------
    tstart: datetime, since this time, the duration is calculated
    Returns
    -------
    out : duration in ms
    """
    te = time.time()
    return f"{int((te - tstart) * 1000)} ms"

def decorator_time(method):
    """
    Decorator function. Show runtime duration for the wrapped function.
    """
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = f"{int((te - ts) * 1000)} ms"
        if 'second' in kw:
            print('%r  %2.2f s' % \
                  (method.__name__, (te - ts))
                  )
        else:
            print('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000)
                  )
        return result
    return timed

def decorator_time_second(method):
    """
    Decorator function. Show runtime duration for the wrapped function.
    """
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        print('%r  %2.2f s' % \
                  (method.__name__, (te - ts))
                  )
        return result

    return timed


def test(**kwargs):
    print(kwargs)



class ttDataBase(object):
      def __init__(self, server_url="http://ixinbuy.com/", loggger=None):
        self.Server_URL = server_url
        self._Device_ID  = ""
        self._Datatype   = 0
        self.Logger = loggger
        
      @property
      def Device_ID(self):
          return self._Device_ID

      @Device_ID.setter
      def Device_ID(self, x):
          self._Device_ID = x

      @property
      def Datatype(self):
          return self._Datatype

      @Datatype.setter
      def Datatype(self, x):
          self._Datatype = x

      @property
      def Logger(self):
          return self._Logger

      @Datatype.setter
      def Logger(self, x):
          self._Logger = None
          loggger = x

          if loggger is not None:
          
             if isinstance(loggger, str):
                self._Logger = logging.getLogger("ttDataBase_logger")
                formatter   = logging.Formatter('%(asctime)s|%(name)s|%(message)s')
                fileHandler = logging.FileHandler(loggger, mode='w')
                fileHandler.setFormatter(formatter)
                streamHandler = logging.StreamHandler()
                streamHandler.setFormatter(formatter)
                level=logging.INFO
                self._Logger.setLevel(level)
                self._Logger.addHandler(fileHandler)
                self._Logger.addHandler(streamHandler)
             elif isinstance(loggger, ttDataLog):
                self._Logger= loggger


      def write_logger(self, *argv,  **kwargs):
        
          if self._Logger is not None:
            
             if isinstance(self._Logger, logging.Logger):
                if len(argv) > 0:
                  self._Logger.info(argv)
             #elif isinstance(self._Logger, ttDataLog):
             else:
                if len(kwargs) > 0:
                  self._Logger.savelog(**kwargs)
               


      def set_client_data(self, **kwargs):  
          for key, value in kwargs.items(): 
              if (str.lower(key) == "datatype"):
                 self.Datatype = value
              elif (str.lower(key) == "device_id"):
                 self.Device_ID = value
              else:
                 print("un-handled client data:", key, value)
               


      def register(self, device_id =None):
          '''
          register a device id with ttData server

          Parameters:
                    device_id (string): Any String to represent a device_id
                
          Returns: 
                    status_code (int): status_code of webservice. 200 means sucessful
          '''
          if device_id is not None:
             self.Device_ID = device_id
                  
          end_point = self.Server_URL + "register"
          payload = {"device_id": self.Device_ID, "category": self.Device_ID}
          
          print("register", payload)
          responce = requests.post(end_point, data = payload)
          print(f"register|{responce.text}")
          #self.logger.info(responce.status_code)
          #self.write_logger(f"register|{responce.text}")
          return (responce.status_code)

      def uploaddata(self, data, dataID = "ID"):
          '''
          '''
          end_point = self.Server_URL + "uploaddata"
          payload = {
                 "data_type": self.Datatype, 
                 "device_id": self.Device_ID,
                 "key"      : dataID,
                 "device_data": data
              }
          print("--- payload: ",   payload)
          responce = requests.post(end_point, json = payload)

          print("--- status: " , responce.status_code, "  ", responce.text)
          print("--- duration",  responce.elapsed.total_seconds())
         

      def get_data(self, **kwargs):
          end_point = self.Server_URL + "getshareddata"
          #save_to_logger(end_point)
          query = { "data_type": self.Datatype, 
                    "device_id": self.Device_ID}
          
          if len(kwargs)>0:
              for key, value in kwargs.items(): 
                  query[key] = value

          print("get_data: ", end_point, query)
         
          try:
              req = requests.get(end_point, params=query)

              if (req.status_code == 200):
                  data = req.json()
                  df = pd.DataFrame(data)
                  return (df)
              else:
                  print("\neror to get_data with status code=", req.status_code)
                  return pd.DataFrame()
          except Exception as err:
                  print("Error handling get_date with {0}".format(err))
                  return pd.DataFrame()
    



class ttDataLog(ttDataBase):
      def __init__(self, server_url="http://ixinbuy.com/", 
                         Device_ID="ttData_Log",
                         clientDataType ="", 
                         clientDeviceID = ""
                   ):
           super().__init__(server_url,None )
           self._clientDataType = clientDataType
           self._clientDeviceID = clientDeviceID
           self.set_client_data(Device_ID = Device_ID, Datatype = 10)
           self.register()


      def savelog(self, name, category,  message, duration, level="INFO"):
        
          namevalue = '{:%Y/%m/%d %I:%M %S.%f}'.format(datetime.now())
          if name != "":
             namevalue = name
    
          data = [{
              "cl_DataType" : self._clientDataType, 
              "cl_DeviceID" : self._clientDeviceID,
              "name"        : namevalue,
              "category"    : category, 
              "level"       : level,
              "message"     : message, 
              "duration"    : duration,
              "ascitime"    : '{:%Y/%m/%d %I:%M %S.%f}'.format(datetime.now()),
              "date"        : '{:%Y-%m-%d}'.format(datetime.now())
          }]
     
          self.uploaddata(data, dataID = "cl_DataType|cl_DeviceID|name")

      def savelog_raw(self, data):
          print("savelog_raw")
          self.uploaddata(data, dataID = "cl_DataType|cl_DeviceID|data_date|name")

class ttDataCovid19(ttDataBase):
      def __init__(self, server_url="http://ixinbuy.com/", logger=None, Device_ID = ""):
           super().__init__(server_url, logger )
           self.set_client_data(Device_ID = Device_ID, Datatype = 2)
           self.register()
        
 
      @staticmethod
      def read_csse_covid_19_daily_reports(Y_M_Ddate):
          '''
          read daily COVID 19 data from github into a pandas dataframe with needed format.

          Parameters:
                    Y_M_Ddate (string): A string represent a date in the format of Y_M_D, for example 2020-03-30
                
          Returns: 
                    dataframe, keycolumn (pandas dataframe, string): pandas dataframe, and the columns as the key needed by ttData
          '''
      
          github_root = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports"
          datetime_object = datetime.strptime(Y_M_Ddate, "%Y-%m-%d")
          date = datetime_object.strftime("%m-%d-%Y")
          ####file = os.path.join(github_root, date + ".csv")
          file = github_root+ "/" + date + ".csv"

          df = pd.read_csv(file, dtype = {'FIPS': str, 'ZIP': str, "ZIP_CODE": str})
        
          df.columns = [x.lower() for x in df.columns]
          df.rename(columns={'province/state': 'province_state',
                   'admin2'            : 'county',
                   'country/region'    : 'country_region',
                   'last update'       : 'last_update',
                   "long_":          "longitude",
                   "long":          "longitude",
                   "lat":           "latitude",
                   "lat_":           "latitude",
                  }, inplace=True)
    
          if ("combined_key" in df.columns):
             df.drop(columns=['combined_key'], inplace=True)
       
          keycolumn = "country_region|province_state|date"

          if ("county" in df.columns):
             keycolumn = "country_region|province_state|county|date"
  
          return (df, keycolumn)

      def upload_data(self, data_date, blocksize=100, waitsec = 1 , numofblocks = 0, logging = False):
          '''
          Upload data into ttData Server through ttData.uploaddata API

          Parameters:
                    data_date (string): A string represent a date in the format of Y_M_D, for example 2020-03-30
                    blocksize (int):    if blocksize >0, the all data will be spliited as multiple batches, this number defines the size of each batch.
                    waitsec   (int):    Used in the multiple batches scenaria, wait <waitsec> between each batch to wait server to process the previous task
                    numofblocks (int):  used by the test stage, num of batches to be processed.
          Returns: 
                    maxStatus_code(int): Status code for web service, 200 means successful.
          '''
          print(f"uploadbatchdata|starting....")

          end_point = self.Server_URL + "uploaddata"
          print(end_point)
          
          start_time = time.time()

          df, key = ttDataCovid19.read_csse_covid_19_daily_reports(data_date)
          errorCount = 0
          i = 0
          totalsize = df.shape[0]

          if (blocksize <= 0) or (blocksize is None):
             blocksize = totalsize
          batches =[list(range(i, min(i+blocksize, totalsize))) for i in range(0,totalsize,blocksize) ]
          maxStatus_code = 200
          blockCount = 0
          print(f"Total Size: {totalsize}\nBatch Size: {blocksize}\nNum of Batches: {numofblocks}")
          if (numofblocks is None) or (numofblocks <=0):
             numofblocks = 10000000
             
          log_data_list = []
          for aBatch in batches:
              json_data_list = []
              for i in aBatch:
                  row = df.iloc[i]
                  #row_as_dic = row.to_dict()
                  device_data = {"date":data_date }
                  device_data.update(json.loads(row.to_json()))
                  json_data_list.append(device_data)
            
              print("#########################################")
              print("\nFor batch from: ", aBatch[0], ", to:", aBatch[-1])
              payload = {
                 "data_type": self.Datatype, 
                 "device_id": self.Device_ID,
                 "key"      : key,
                 "device_data": json_data_list
              }
             
              #print(payload)
              responce = requests.post(end_point, json = payload)
              print(responce.status_code, "  ", responce.text)
              v_duration = responce.elapsed.total_seconds()
              v_message = f"\"'date': {data_date}, 'from': {aBatch[0]}, 'to': {aBatch[-1]}\""

              print("duration:", v_duration)
              if responce.status_code != 200:
                 print(responce.text)
              maxStatus_code = max(maxStatus_code, responce.status_code)

              log_data_list.append(
                  {
                      "data_date": data_date,
                      "name"     :  '{:%Y/%m/%d %I:%M %S.%f}'.format(datetime.now()),
                      "batch_id"    : f"{aBatch[0]}:{aBatch[-1]}",
                      "api_duration": v_duration,
                      "api_r_status":  responce.status_code,
                      "api_r_text"  :  responce.text,
                      "total_size"  :  totalsize,
                      "client_data_type": self.Datatype,
                      "client_device_id": self.Device_ID
                  }
              )

              time.sleep(waitsec)
              blockCount += 1

              if (blockCount >= numofblocks):
                 break
          
          if logging == True:
                try:
                     print("logging....")
                     self._Logger.savelog_raw(log_data_list)
                     print("logged....")
                except Exception:
                     print("logging failed")

          return (totalsize, maxStatus_code)

      

def init_argparse() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser(
        usage="%(prog)s [OPTION] [FILE]...",
        description="load COVID19 daily data to ttData via web API.",
        epilog='Enjoy the program! :)'
    )
    parser.add_argument(
        "-s", "--s", action='store', 
        help="ttData Server URL",
        required = True)
    parser.add_argument(
        "-d", "--d", action='store', 
        help="device_id, required",
        required = True)
    parser.add_argument(
        "-t", "--t", 
        action='store', 
        type=str,
        help="date in the format of YYYY-MM-DD",
        required = True)
    parser.add_argument(
        "-l", "--l", 
        action='store', 
        type=str,
        help="runtime logfile")
    
    return parser

from datetime import date, timedelta

def generate(sdate, edate):
    delta = edate - sdate       # as timedelta
    return [ '{:%Y-%m-%d}'.format(sdate + timedelta(days=i)) for i in range(delta.days + 1)]

@decorator_time_second
def main(interactive=False) -> None:
    server_url="http://ttdata.life:7061/"
    device = "XDCOVID19V70"
    log_file="ttDataBaseTest00.txt"
    sdate = date(2020, 4, 1)   # start date
    edate = date(2020, 4, 30)   # end date
    times = generate(sdate, edate)
    ##times    = ["2020-04-26", "2020-04-27", "2020-04-28"]
    if (interactive == False):
       parser = init_argparse()
       args = parser.parse_args()
       server_url = args.s
       device = args.d
       time = args.t
       log_file = args.l
    
    my_ttDataLog = ttDataLog(server_url=server_url,  Device_ID=device + "_log", 
                         clientDataType =2, clientDeviceID = device)

    my_ttData = ttDataCovid19(server_url=server_url, logger=my_ttDataLog, Device_ID = device)

    for tt in times:
       print("\n===", tt)
       (cnt, status) = my_ttData.upload_data(tt, blocksize=300, numofblocks=0, logging=True)
       print(f"{cnt}, Final status: {status}")
    
    show_get_data = False
    if show_get_data:
        try:
            df = my_ttDataLog.get_data()
            print(df)
        except Exception:
            print("get_data failed")


if __name__ == '__main__':
    try:
      main(interactive=True)
      print("\n====================\nmain() Finished")
    except Exception:
      print("Exception in main()")
      exit(1)
          

register {'device_id': 'XDCOVID19V70_log', 'category': 'XDCOVID19V70_log'}
register|device: XDCOVID19V70_log is already registered
register {'device_id': 'XDCOVID19V70', 'category': 'XDCOVID19V70'}
register|device: XDCOVID19V70 is already registered

=== 2020-04-27
uploadbatchdata|starting....
http://ttdata.life:7061/uploaddata
Total Size: 3152
Batch Size: 100
Num of Batches: 3
#########################################

For batch from:  0 , to: 99
200    COVID19 data uploaded successfully
duration: 0.460019
#########################################

For batch from:  100 , to: 199
200    COVID19 data uploaded successfully
duration: 5.389224
#########################################

For batch from:  200 , to: 299
200    COVID19 data uploaded successfully
duration: 5.274929
logging....
savelog_raw
--- payload:  {'data_type': 10, 'device_id': 'XDCOVID19V70_log', 'key': 'cl_DataType|cl_DeviceID|data_date|name', 'device_data': [{'data_date': '2020-04-27', 'name': '2020/05/25 03:57 08.211684

### Testing log features

In [4]:
import requests
import json
from datetime import datetime
URL = baseURL +"register"

payload = {"device_id":DEVICE_ID+"_log"}

#Passing payload as dict
responce = requests.post(URL, data = payload)
print(responce.status_code)
print(responce.text)

URL = baseURL + "uploaddata"
print(URL)

payload = {
    "data_type": 11, 
    "device_id": DEVICE_ID +"_log", 
    "key": "cl_DataType|cl_DeviceID|ID",
    "device_data": 
    [
     {        "cl_DataType" : 2, 
              "cl_DeviceID" : DEVICE_ID,
              "ID"          : '{:%Y/%m/%d - %I:%M %S.%f}'.format(datetime.now()),
              "Category"    : "INFO", 
              "Message"     : "Test Log", 
              "Duration"    : 10,
              "RAM"         : 12,
              "CPU"         : 90,
              "TimeStamp"   : '{:%Y/%m/%d %I:%M %S.%f}'.format(datetime.now()),
              "date"        : '{:%Y-%m-%d}'.format(datetime.now())
     },
     {        "cl_DataType" : 2, 
              "cl_DeviceID" : DEVICE_ID,
              "ID"          : '{:%Y/%m/%d - %I:%M %S.%f}'.format(datetime.now()),
              "Category"    : "ERROR", 
              "Message"     : "Test Log", 
              "Duration"    : 10,
              "RAM"         : 1,
              "CPU"         : 22,
              "TimeStamp"   : '{:%Y/%m/%d %I:%M %S.%f}'.format(datetime.now()),
              "date"        : '{:%Y-%m-%d}'.format(datetime.now())
     }
    ]
}

print(payload)


#Passing payload as dict
import time
import pandas as pd
ts = time.time()
responce = requests.post(URL, json = payload)
print(responce.status_code)
print(responce.text)
te = time.time()
print('%s  %2.2f sec' % ("uploaddata", (te - ts) ))

query = {"data_type": 10, "device_id":DEVICE_ID + "_log" } ####, "from_date": "2020-04-02", "to_date":  "2020-06-03"}

req = requests.get(baseURL + "getshareddata", params=query)
print(req.url)
print(req.status_code)

print(req.text)
data = req.json()
df = pd.DataFrame(data)

df

422
category is required
http://ttdata.life:7061/uploaddata
{'data_type': 11, 'device_id': 'XDCOVID19V5_log', 'key': 'cl_DataType|cl_DeviceID|ID', 'device_data': [{'cl_DataType': 2, 'cl_DeviceID': 'XDCOVID19V5', 'ID': '2020/05/25 - 12:43 34.556784', 'Category': 'INFO', 'Message': 'Test Log', 'Duration': 10, 'RAM': 12, 'CPU': 90, 'TimeStamp': '2020/05/25 12:43 34.556823', 'date': '2020-05-25'}, {'cl_DataType': 2, 'cl_DeviceID': 'XDCOVID19V5', 'ID': '2020/05/25 - 12:43 34.556835', 'Category': 'ERROR', 'Message': 'Test Log', 'Duration': 10, 'RAM': 1, 'CPU': 22, 'TimeStamp': '2020/05/25 12:43 34.556840', 'date': '2020-05-25'}]}
200
data uploaded successfully
uploaddata  1.84 sec
http://ttdata.life:7061/getshareddata?data_type=10&device_id=XDCOVID19V5_log
200
[{"cl_DataType":2,"cl_DeviceID":"XDCOVID19V5","ID":"2020/05/23 - 03:57 17.016183","Category":"INFO","Message":"Test Log","Duration":10,"RAM":12,"CPU":90,"timestamp":"2020/05/23 - 03:57 17.016217","valid":true},{"cl_DataType":2,"cl_Devi

,cl_DataType,cl_DeviceID,ID,Category,Message,Duration,RAM,CPU,timestamp,valid,Date,TimeStamp,date
0,2,XDCOVID19V5,2020/05/23 - 03:57 17.016183,INFO,Test Log,10,12,90,2020/05/23 - 03:57 17.016217,True,NaN,NaN,NaN
1,2,XDCOVID19V5,2020/05/23 - 03:57 17.016223,ERROR,Test Log,10,1,22,2020/05/23 - 03:57 17.016228,True,NaN,NaN,NaN
2,2,XDCOVID19V5,2020/05/23 - 03:58 25.041203,INFO,Test Log,10,12,90,2020/05/23 - 03:58 25.041239,True,NaN,NaN,NaN
3,2,XDCOVID19V5,2020/05/23 - 03:58 25.041245,ERROR,Test Log,10,1,22,2020/05/23 - 03:58 25.041249,True,NaN,NaN,NaN
4,2,XDCOVID19V5,2020/05/23 - 03:59 08.335369,INFO,Test Log,10,12,90,2020/05/23 - 03:59 08.335403,True,NaN,NaN,NaN
5,2,XDCOVID19V5,2020/05/23 - 03:59 08.335410,ERROR,Test Log,10,1,22,2020/05/23 - 03:59 08.335414,True,NaN,NaN,NaN
6,2,XDCOVID19V5,2020/05/23 - 03:59 23.309416,INFO,Test Log,10,12,90,2020/05/23 - 03:59 23.309447,True,NaN,NaN,NaN
7,2,XDCOVID19V5,2020/05/23 - 03:59 23.309453,ERROR,Test Log,10,1,22,2020/05/23 - 03:59 23.309457,True,NaN,NaN,NaN
8,2,XDCOVID19V5,2020/05/23 - 03:59 44.116016,INFO,Test Log,10,12,90,2020/05/23 - 03:59 44.116058,True,NaN,NaN,NaN
9,2,XDCOVID19V5,2020/05/23 - 03:59 44.116068,ERROR,Test Log,10,1,22,2020/05/23 - 03:59 44.116075,True,NaN,NaN,NaN


In [31]:
from datetime import date, timedelta

sdate = date(2008, 8, 15)   # start date
edate = date(2008, 8, 16)   # end date

def generate(sdate, edate):
    delta = edate - sdate       # as timedelta
    return [ '{:%Y-%m-%d}'.format(sdate + timedelta(days=i)) for i in range(delta.days + 1)]

lst = generate(sdate, edate)
print(lst)

['2008-08-15', '2008-08-16']
